In [3]:
from katabatic.models.TableGAN import TableGANAdapter, TableGAN, preprocess_data, postprocess_data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Initialize the adapter with a specific privacy setting
tablegan_adapter = TableGANAdapter(type='continuous', privacy_setting='high')
data_path = 'data/Nursery/Nursery.csv'
df = pd.read_csv(data_path)
labelencoder=preprocessing.LabelEncoder()
df= df.apply(lambda col: labelencoder.fit_transform(col) if col.dtype =='object' else col)

In [4]:
print(df.info())
print(df.isnull().sum())
print(df.isin(['?']).sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0    parents  12960 non-null  int32
 1   has_nurs  12960 non-null  int32
 2   form      12960 non-null  int32
 3   children  12960 non-null  int32
 4   housing   12960 non-null  int32
 5   finance   12960 non-null  int32
 6   social    12960 non-null  int32
 7   health    12960 non-null  int32
 8   Target    12960 non-null  int32
dtypes: int32(9)
memory usage: 455.8 KB
None
 parents    0
has_nurs    0
form        0
children    0
housing     0
finance     0
social      0
health      0
Target      0
dtype: int64
 parents    0
has_nurs    0
form        0
children    0
housing     0
finance     0
social      0
health      0
Target      0
dtype: int64


In [ ]:
x = df.copy().drop('Target', axis=1)
y = df['Target']

x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
tablegan_adapter.fit(x_train, y_train, epochs=50, batch_size=64)

---FIT TableGAN Model with high privacy setting
---Initialise TableGAN Model


d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10/50: [D loss: -1.6309] [G loss: 1.0728] [C loss: 0.2399]
Epoch 20/50: [D loss: -1.5167] [G loss: 1.5420] [C loss: 0.1495]
Epoch 30/50: [D loss: -1.5107] [G loss: 2.3895] [C loss: 0.0849]
Epoch 40/50: [D loss: -1.1561] [G loss: 3.0408] [C loss: 0.0544]
Epoch 50/50: [D loss: -1.1526] [G loss: 3.1096] [C loss: 0.0345]


In [7]:
# Generate synthetic data
synthetic_data = tablegan_adapter.generate(size=1000)

---Generate from TableGAN Model


In [8]:
synthetic_df = pd.DataFrame(synthetic_data)

x_sync_train = synthetic_df.drop(synthetic_df.columns[-1],axis=1).values
y_sync_train = synthetic_df.iloc[ :, -1:].values

In [13]:
# TSTR (train synthetic test real)
tstr_score_lr  = LogisticRegression().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_rf  = RandomForestClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_mlp = MLPClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)
y_sync_train = LabelEncoder().fit_transform(y_sync_train)
xgbt_classifier = XGBClassifier(eval_metric='logloss' )
tstr_score_xgbt = xgbt_classifier.fit(x_sync_train, y_sync_train).score(x_test, y_test)


# TRTR (train real test real)
trtr_score_lr  = LogisticRegression().fit(x_train, y_train).score(x_test, y_test)
trtr_score_rf  = RandomForestClassifier().fit(x_train, y_train).score(x_test, y_test)
trtr_score_mlp = MLPClassifier().fit(x_train, y_train).score(x_test, y_test)
y_train = LabelEncoder().fit_transform(y_train)
xgbt_classifier = XGBClassifier(eval_metric='logloss')
trtr_score_xgbt = xgbt_classifier.fit(x_train, y_train).score(x_test, y_test)
df_evaluate = pd.DataFrame([
    ['TSTR', tstr_score_lr, tstr_score_rf, tstr_score_mlp, tstr_score_xgbt],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp, trtr_score_xgbt]
], columns=['Evaluated Item', 'LR', 'RF', 'MLP', 'XGBT'])
print(df_evaluate)

d:\Anaconda3\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stoch

  Evaluated Item        LR        RF       MLP      XGBT
0           TSTR  0.288194  0.325231  0.303241  0.374614
1           TRTR  0.762346  0.983025  0.985340  0.672454
